In [1]:
!wget https://raw.githubusercontent.com/iamsli/cs230-final-project/main/data/en-valid-all-test.txt?token=AHEDSMEMC4HBI3P3RF7SLJLAVAXN6

--2021-05-15 03:21:05--  https://raw.githubusercontent.com/iamsli/cs230-final-project/main/data/en-valid-all-test.txt?token=AHEDSMEMC4HBI3P3RF7SLJLAVAXN6
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5565536 (5.3M) [text/plain]
Saving to: ‘en-valid-all-test.txt?token=AHEDSMEMC4HBI3P3RF7SLJLAVAXN6’

en-valid-all-test.t 100%[===================>]   5.31M  3.32MB/s    in 1.6s    

2021-05-15 03:21:07 (3.32 MB/s) - ‘en-valid-all-test.txt?token=AHEDSMEMC4HBI3P3RF7SLJLAVAXN6’ saved [5565536/5565536]



In [2]:
!wget https://raw.githubusercontent.com/iamsli/cs230-final-project/main/data/en-valid-all-train.txt?token=AHEDSMAFRNHAYWV7ZE3NHATAVAYAE

--2021-05-15 03:21:07--  https://raw.githubusercontent.com/iamsli/cs230-final-project/main/data/en-valid-all-train.txt?token=AHEDSMAFRNHAYWV7ZE3NHATAVAYAE
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25044403 (24M) [text/plain]
Saving to: ‘en-valid-all-train.txt?token=AHEDSMAFRNHAYWV7ZE3NHATAVAYAE’

en-valid-all-train. 100%[===================>]  23.88M  6.21MB/s    in 5.6s    

2021-05-15 03:21:13 (4.30 MB/s) - ‘en-valid-all-train.txt?token=AHEDSMAFRNHAYWV7ZE3NHATAVAYAE’ saved [25044403/25044403]



In [3]:
!mv en-valid-all-train.txt?token=AHEDSMAFRNHAYWV7ZE3NHATAVAYAE en-valid-all-train.txt
!mv en-valid-all-test.txt?token=AHEDSMEMC4HBI3P3RF7SLJLAVAXN6 en-valid-all-test.txt

In [4]:
import re
import string
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('using device: {}'.format(device))

using device: cpu


In [5]:
batch_size = 32
epochs = 10
lr = 0.003

In [6]:
# def additional_preprocess(raw_data):
#   full_data = []
#   for i in raw_data:
#     i = i.lower()
#     i = i.replace('\n')
#     i = i.translate(str.maketrans('', '', string.punctuation))
#     if '\t' in i:
#       tmp = 

In [7]:
with open('en-valid-all-test.txt', 'r') as f:
  raw_test = f.readlines()

with open('en-valid-all-train.txt', 'r') as f:
  raw_train = f.readlines()

def get_mapping(sentence):
  indices = [word_to_idx[i] for i in sentence]
  return torch.tensor(indices)

def get_vocab(raw_list):
  dataset = []
  for i in raw_list:
    i = i.lower()
    i = i.replace('\t', '')
    i = i.replace('\n', '')
    i = i.translate(str.maketrans('', '', string.punctuation))

    dataset.extend(i.split(' '))
  dataset = list(set(dataset))

  #add 1 to never get 0
  word_to_idx = {w: i+1 for i, w in enumerate(dataset)}
  idx_to_word = {i+1: w for i, w in enumerate(dataset)}

  return word_to_idx, idx_to_word

word_to_idx, idx_to_word = get_vocab(raw_train + raw_test)
vocab_size = len(word_to_idx)

In [8]:
def get_longest_sentence(raw_data):
  longest = 0
  for i in raw_data:
    curr_len = len(i.split(' '))
    if curr_len > longest:
      longest = curr_len

  return longest

longest_sentence = get_longest_sentence(raw_train + raw_test)

In [9]:
def get_dataset(raw_data, label_number = 1):
  inputs = []
  outputs = []
  for i in raw_data:
    i = i.lower()
    i = i.replace('\t', '')
    i = i.replace('\n', '')
    i = i.translate(str.maketrans('', '', string.punctuation))

    tmp = i.split(' ')
    string_so_far = []
    for index, data in enumerate(tmp):
      if not index + label_number == len(tmp):
        string_so_far.append(data)
        encoding = get_mapping(string_so_far)
        zeros_length = longest_sentence - len(string_so_far)
        padded_encoding = torch.cat((encoding, torch.zeros(zeros_length)))
        inputs.append(padded_encoding)
        outputs.append(get_mapping(tmp[index + 1:index + label_number + 1]))

  ds = TensorDataset(torch.stack(inputs), torch.stack(outputs))
  return ds

In [40]:
train_dataset = get_dataset(raw_train, 1)

In [41]:
len(train_dataset)

4498583

In [11]:
train_loader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True)

In [35]:
class generator(nn.Module):
  def __init__(self, embedding_dim, hidden_dim, vocab_size, longest_sentence, output_size = 1):
    super(generator, self).__init__()
    self.embeddings = nn.Embedding(vocab_size, embedding_dim)
    self.lstm = nn.LSTM(embedding_dim, hidden_dim)
    self.linear = nn.Linear(hidden_dim * longest_sentence, vocab_size)

  def forward(self, input_sentence):
    embeds = self.embeddings(input_sentence)
    lstm_out, _ = self.lstm(embeds)
    output = self.linear(lstm_out.view(input_sentence.shape[0], -1))
    output = F.softmax(output, dim = 0)
    return output

In [42]:
model = generator(128, 128, vocab_size, longest_sentence, output_size = 1)
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = lr)

In [44]:
for i in range(epochs):
  for index, data in enumerate(train_loader):
    optimizer.zero_grad()
    x = data[0].to(device)
    y = data[1].to(device)

    prediction = model(x.long())
    loss = criterion(prediction.squeeze(), y.squeeze())
    loss.backward()
    optimizer.step()